In [2]:
# from google.colab import drive
# fsaaffdrive.mount('/content/drive')

In [3]:
!pip install gensim==4.3.2
!pip install texthero==1.0.8



In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap

# Torch ML libraries
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader

# Misc.
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Set intial variables and constants
%config InlineBackend.figure_format='retina'

# Graph Designs
sns.set(style='whitegrid', palette='muted', font_scale=1.2)
HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]
sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))
rcParams['figure.figsize'] = 12, 8

# Random seed for reproducibilty
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

# Set GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
print(torch.cuda.is_available())

True


In [ ]:
df1_des = pd.read_csv("./Data/bug_description_2022-05-09.csv")
df2_detail = pd.read_csv("./Data/bug_2022-05-09.csv")
print(df1_des.head())
print(df2_detail.head())

   bug_id                                        description
0      36  Created by J. Daniel Powell (dan@java-linux.or...
1     117  Created by Thorsten Heit (thorsten@wurzelausix...
2     240  Created by Dan McGuirk (mcguirk@indirect.com) ...
3     244  Created by Keith Adams (kma@cs.brown.edu) on T...
4     260  Created by Ralf Hildebrandt (R.Hildebrandt@tu-...
    id                                            summary    status  \
0   36              Floating Point Exception on Execution  RESOLVED   
1  117  Compiling Navigator on SPARC/Linux doesn't com...  VERIFIED   
2  240         XP_GetBuiltinString error messages missing  VERIFIED   
3  244                                   Busted -MDupdate  VERIFIED   
4  260            HTML pages or images don't get diplayed  VERIFIED   

  resolution platform     component    product keywords version   op_sys  ...  \
0    INVALID      x86  Build Config  SeaMonkey       []   Trunk    Linux  ...   
1      FIXED      x86  Build Config  SeaMonke

In [ ]:
print(df2_detail["resolution"].unique(),"\n")
print(df2_detail["status"].unique(),"\n")
print(df2_detail["priority"].unique(),"\n")


['INVALID' 'FIXED' 'WONTFIX' 'DUPLICATE' 'WORKSFORME' 'EXPIRED'
 'INCOMPLETE' 'MOVED' 'INACTIVE'] 

['RESOLVED' 'VERIFIED' 'CLOSED'] 

['P2' 'P3' 'P1' 'P5' 'P4' '--'] 



In [ ]:
import texthero as hero

df1_des["description"] = hero.clean(df1_des["description"])
print(df1_des["description"])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


0        created j daniel powell dan java linux org mon...
1        created thorsten heit thorsten wurzelausix cs ...
2        created dan mcguirk mcguirk indirect com thurs...
3        created keith adams kma cs brown edu thursday ...
4        created ralf hildebrandt r hildebrandt tu bs d...
                               ...                        
96271    user agent mozilla windows nt win64 x64 rv gec...
96272    created attachment x png user agent mozilla wi...
96273    user agent mozilla windows nt win64 x64 rv gec...
96274    user agent mozilla windows nt win64 x64 rv gec...
96275    user agent mozilla windows nt win64 x64 applew...
Name: description, Length: 96276, dtype: object


In [ ]:
print(df1_des.shape)

(96276, 2)


In [ ]:
from transformers import pipeline
data = df1_des["description"].head(10).tolist()
ids = df1_des["bug_id"].head(10).tolist()
print(type(data))
tokenizer_kwargs = {'padding':True,'truncation':True,'max_length':512}
specific_model = pipeline(model="cardiffnlp/twitter-roberta-base-sentiment-latest",device=0,**tokenizer_kwargs)

results = specific_model(data)
for result in results:
  result["id"] = ids.pop(0)
print(results)
print(type(specific_model))

<class 'list'>


config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

[{'label': 'neutral', 'score': 0.9295428395271301, 'id': 36}, {'label': 'neutral', 'score': 0.9124292135238647, 'id': 117}, {'label': 'neutral', 'score': 0.7174078822135925, 'id': 240}, {'label': 'neutral', 'score': 0.9394207000732422, 'id': 244}, {'label': 'neutral', 'score': 0.8545593023300171, 'id': 260}, {'label': 'neutral', 'score': 0.9204272031784058, 'id': 299}, {'label': 'neutral', 'score': 0.8482767343521118, 'id': 301}, {'label': 'neutral', 'score': 0.8987032175064087, 'id': 302}, {'label': 'neutral', 'score': 0.878762423992157, 'id': 303}, {'label': 'neutral', 'score': 0.8690948486328125, 'id': 424}]
<class 'transformers.pipelines.text_classification.TextClassificationPipeline'>


In [ ]:
from transformers import pipeline,AutoTokenizer
data = df1_des["description"].head(300).tolist()
ids = df1_des["bug_id"].head(300).tolist()


MODEL_NAME = "cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer_kwargs = {'padding':True,'truncation':True,'max_length':512}
specific_model = pipeline(model=MODEL_NAME,tokenizer=tokenizer,device=0,**tokenizer_kwargs)
results = specific_model(data)
for result in results:
  result["id"] = ids.pop(0)
print(results)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'label': 'neutral', 'score': 0.9295428395271301, 'id': 36}, {'label': 'neutral', 'score': 0.9124292135238647, 'id': 117}, {'label': 'neutral', 'score': 0.7174078822135925, 'id': 240}, {'label': 'neutral', 'score': 0.9394207000732422, 'id': 244}, {'label': 'neutral', 'score': 0.8545593023300171, 'id': 260}, {'label': 'neutral', 'score': 0.9204272031784058, 'id': 299}, {'label': 'neutral', 'score': 0.8482767343521118, 'id': 301}, {'label': 'neutral', 'score': 0.8987032175064087, 'id': 302}, {'label': 'neutral', 'score': 0.878762423992157, 'id': 303}, {'label': 'neutral', 'score': 0.8690948486328125, 'id': 424}, {'label': 'neutral', 'score': 0.6748421788215637, 'id': 426}, {'label': 'neutral', 'score': 0.92470782995224, 'id': 491}, {'label': 'neutral', 'score': 0.9345173239707947, 'id': 515}, {'label': 'neutral', 'score': 0.7458454966545105, 'id': 704}, {'label': 'neutral', 'score': 0.5845586657524109, 'id': 739}, {'label': 'negative', 'score': 0.6204494833946228, 'id': 781}, {'label': 

In [ ]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 4.8 MB/s eta 0:00:00


In [ ]:
!pip show accelerate

Name: accelerate
Version: 0.26.1
Summary: Accelerate
Home-page: https://github.com/huggingface/accelerate
Author: The HuggingFace team
Author-email: sylvain@huggingface.co
License: Apache
Location: /usr/local/lib/python3.10/dist-packages
Requires: huggingface-hub, numpy, packaging, psutil, pyyaml, safetensors, torch
Required-by: 


In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.9 MB/s eta 0:00:00


In [ ]:

# from datasets import load_dataset
# from transformers import LongformerTokenizer
# from transformers import LongformerForSequenceClassification, Trainer, TrainingArguments

# tweeteval_dataset = load_dataset("tweet_eval", "sentiment")
# print(tweeteval_dataset['test'][:100])


# tokenizer = LongformerTokenizer.from_pretrained("allenai/longformer-base-4096")

# def tokenize_batch(batch):
#     return tokenizer(batch["text"], padding="max_length", truncation=True)

# tokenized_dataset = tweeteval_dataset.map(tokenize_batch, batched=True)



# model = LongformerForSequenceClassification.from_pretrained("allenai/longformer-base-4096")

# training_args = TrainingArguments(
#     output_dir="./sentiment_finetuned_model",
#     per_device_train_batch_size=8,
#     num_train_epochs=3,
#     save_steps=500,
#     save_total_limit=2,
# )

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized_dataset["train"],
# )

# trainer.train()
# eval_results = trainer.evaluate(eval_dataset=tokenized_dataset["validation"])
# print(eval_results)


# model.save_pretrained("./sentiment_finetuned_model")
# tokenizer.save_pretrained("./sentiment_finetuned_model")



In [ ]:
# from transformers import LongformerForSequenceClassification, LongformerTokenizer

# model = LongformerForSequenceClassification.from_pretrained("./sentiment_finetuned_model")
# tokenizer = LongformerTokenizer.from_pretrained("./sentiment_finetuned_model")

# text = "I love you."
# inputs = tokenizer(text, return_tensors="pt")
# outputs = model(**inputs)
# predictions = outputs.logits.argmax(dim=1)

# print("Sentiment:", predictions.item())

In [ ]:
from transformers import pipeline,AutoTokenizer
data = df1_des["description"].head(10000).tolist()
ids = df1_des["bug_id"].head(10000).tolist()


MODEL_NAME = "cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer_kwargs = {'padding':True,'truncation':True,'max_length':512}
model = pipeline(model=MODEL_NAME,tokenizer=tokenizer,**tokenizer_kwargs)
results = model(data)
for result in results:
  result["id"] = ids.pop(0)
print(results)
non_neutrals = []
negatives=[]
positives=[]
for result in results:
  if result["label"] != "neutral":
    non_neutrals.append(result)
  if result["label"] == "negative":
    negatives.append(result)
  if result["label"] == "positive":
    positives.append(result)
print("the number of non-neutral feedbacks:",len(non_neutrals))
print("the number of negative feedbacks:",len(negatives))
print("the number of positive feedbacks:",len(positives))
print(non_neutrals)
print(negatives)
print(positives)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'label': 'neutral', 'score': 0.9295428395271301, 'id': 36}, {'label': 'neutral', 'score': 0.9124292135238647, 'id': 117}, {'label': 'neutral', 'score': 0.7174075245857239, 'id': 240}, {'label': 'neutral', 'score': 0.9394207000732422, 'id': 244}, {'label': 'neutral', 'score': 0.8545593023300171, 'id': 260}, {'label': 'neutral', 'score': 0.9204272031784058, 'id': 299}, {'label': 'neutral', 'score': 0.84827721118927, 'id': 301}, {'label': 'neutral', 'score': 0.8987032771110535, 'id': 302}, {'label': 'neutral', 'score': 0.8787622451782227, 'id': 303}, {'label': 'neutral', 'score': 0.8690947890281677, 'id': 424}, {'label': 'neutral', 'score': 0.6748405694961548, 'id': 426}, {'label': 'neutral', 'score': 0.92470782995224, 'id': 491}, {'label': 'neutral', 'score': 0.9345172643661499, 'id': 515}, {'label': 'neutral', 'score': 0.7458454370498657, 'id': 704}, {'label': 'neutral', 'score': 0.5845575928688049, 'id': 739}, {'label': 'negative', 'score': 0.6204494833946228, 'id': 781}, {'label': '

In [ ]:
df3_bughistory = pd.read_csv("./Data/bug_history_2022-05-09.csv",lineterminator='\n')
fixed_resolution = df3_bughistory.loc[(df3_bughistory["changed_field"]=="resolution") & (pd.isnull(df3_bughistory["old_value"])) & (df3_bughistory['new_value']=="FIXED"),["bug_id","when","changed_field","old_value","new_value"]].drop_duplicates(subset=["bug_id"])
print(fixed_resolution)


          bug_id                  when changed_field old_value new_value
18           117  2001-01-06T06:01:07Z    resolution       NaN     FIXED
83           424  1998-10-20T20:59:12Z    resolution       NaN     FIXED
128          838  1999-03-12T01:07:24Z    resolution       NaN     FIXED
143          883  1999-03-27T00:59:55Z    resolution       NaN     FIXED
161          909  1999-03-13T15:59:27Z    resolution       NaN     FIXED
...          ...                   ...           ...       ...       ...
1023660  1714311  2021-06-21T22:20:32Z    resolution       NaN     FIXED
1023711  1714345  2021-06-03T15:10:51Z    resolution       NaN     FIXED
1023773  1714443  2021-06-09T06:24:34Z    resolution       NaN     FIXED
1023804  1714461  2021-06-25T09:37:34Z    resolution       NaN     FIXED
1023830  1714492  2021-06-04T10:54:52Z    resolution       NaN     FIXED

[19554 rows x 5 columns]


In [ ]:
created_time = df2_detail[df2_detail["id"].isin(fixed_resolution["bug_id"])]
print(created_time[['id',"creation_time"]])

            id creation_time
1          117    1998-04-08
9          424    1998-06-06
16         838    1998-09-18
17         883    1998-09-24
19         909    1998-09-25
...        ...           ...
88133  1714183    2021-06-02
88142  1714248    2021-06-03
88144  1714287    2021-06-03
88148  1714311    2021-06-03
88160  1714461    2021-06-03

[15475 rows x 2 columns]


In [ ]:
fixed_resolution = fixed_resolution[fixed_resolution["bug_id"].isin(created_time["id"])]
print(fixed_resolution)

          bug_id                  when changed_field old_value new_value
18           117  2001-01-06T06:01:07Z    resolution       NaN     FIXED
83           424  1998-10-20T20:59:12Z    resolution       NaN     FIXED
128          838  1999-03-12T01:07:24Z    resolution       NaN     FIXED
143          883  1999-03-27T00:59:55Z    resolution       NaN     FIXED
161          909  1999-03-13T15:59:27Z    resolution       NaN     FIXED
...          ...                   ...           ...       ...       ...
1023519  1714183  2021-06-10T11:45:24Z    resolution       NaN     FIXED
1023587  1714248  2021-06-18T04:20:09Z    resolution       NaN     FIXED
1023626  1714287  2021-06-24T10:43:35Z    resolution       NaN     FIXED
1023660  1714311  2021-06-21T22:20:32Z    resolution       NaN     FIXED
1023804  1714461  2021-06-25T09:37:34Z    resolution       NaN     FIXED

[15475 rows x 5 columns]


In [ ]:
print(results)
col1_id = [result["id"] for result in results]
col2_label = [result["label"] for result in results]
col3_score = [result["score"] for result in results]
labeled_data_dict = {"id":col1_id,"label":  col2_label,"score":col3_score}
labeled_df = pd.DataFrame.from_dict(labeled_data_dict)
print(labeled_df)

[{'label': 'neutral', 'score': 0.9295428395271301, 'id': 36}, {'label': 'neutral', 'score': 0.9124292135238647, 'id': 117}, {'label': 'neutral', 'score': 0.7174075245857239, 'id': 240}, {'label': 'neutral', 'score': 0.9394207000732422, 'id': 244}, {'label': 'neutral', 'score': 0.8545593023300171, 'id': 260}, {'label': 'neutral', 'score': 0.9204272031784058, 'id': 299}, {'label': 'neutral', 'score': 0.84827721118927, 'id': 301}, {'label': 'neutral', 'score': 0.8987032771110535, 'id': 302}, {'label': 'neutral', 'score': 0.8787622451782227, 'id': 303}, {'label': 'neutral', 'score': 0.8690947890281677, 'id': 424}, {'label': 'neutral', 'score': 0.6748405694961548, 'id': 426}, {'label': 'neutral', 'score': 0.92470782995224, 'id': 491}, {'label': 'neutral', 'score': 0.9345172643661499, 'id': 515}, {'label': 'neutral', 'score': 0.7458454370498657, 'id': 704}, {'label': 'neutral', 'score': 0.5845575928688049, 'id': 739}, {'label': 'negative', 'score': 0.6204494833946228, 'id': 781}, {'label': '

In [ ]:
fixed_resolution = fixed_resolution.head(10000)
created_time = created_time[["id","creation_time"]]
created_time = created_time[created_time["id"].isin(fixed_resolution['bug_id'])]
print(created_time)
print(fixed_resolution)

           id creation_time
1         117    1998-04-08
9         424    1998-06-06
16        838    1998-09-18
17        883    1998-09-24
19        909    1998-09-25
...       ...           ...
60561  482966    2009-03-12
60595  483354    2009-03-14
60598  483363    2009-03-14
60608  483443    2009-03-14
60622  483505    2009-03-15

[10000 rows x 2 columns]
        bug_id                  when changed_field old_value new_value
18         117  2001-01-06T06:01:07Z    resolution       NaN     FIXED
83         424  1998-10-20T20:59:12Z    resolution       NaN     FIXED
128        838  1999-03-12T01:07:24Z    resolution       NaN     FIXED
143        883  1999-03-27T00:59:55Z    resolution       NaN     FIXED
161        909  1999-03-13T15:59:27Z    resolution       NaN     FIXED
...        ...                   ...           ...       ...       ...
643023  482966  2009-05-13T18:53:35Z    resolution       NaN     FIXED
643266  483354  2009-03-16T17:53:04Z    resolution       NaN     FIXED

In [ ]:
create_fixed = pd.merge(fixed_resolution,created_time,left_on = "bug_id",right_on="id",how="left")[["bug_id","creation_time","when"]].rename(columns = {'when':'fixed_time'})

create_fixed["fixed_time"] = pd.to_datetime(create_fixed["fixed_time"]).dt.strftime('%Y-%m-%d')
create_fixed["fixed_time"] = pd.to_datetime(create_fixed["fixed_time"])

create_fixed["creation_time"] = pd.to_datetime(create_fixed["creation_time"])
create_fixed["response_time(days)"] = create_fixed["fixed_time"] - create_fixed["creation_time"]

create_fixed["response_time(days)"] = create_fixed["response_time(days)"].dt.days
print(create_fixed)

      bug_id creation_time fixed_time  response_time(days)
0        117    1998-04-08 2001-01-06                 1004
1        424    1998-06-06 1998-10-20                  136
2        838    1998-09-18 1999-03-12                  175
3        883    1998-09-24 1999-03-27                  184
4        909    1998-09-25 1999-03-13                  169
...      ...           ...        ...                  ...
9995  482966    2009-03-12 2009-05-13                   62
9996  483354    2009-03-14 2009-03-16                    2
9997  483363    2009-03-14 2010-10-07                  572
9998  483443    2009-03-14 2009-03-14                    0
9999  483505    2009-03-15 2013-07-16                 1584

[10000 rows x 4 columns]


In [ ]:
labeled_df = labeled_df[labeled_df["id"].isin(create_fixed["bug_id"])]
print(labeled_df)
print("number of neutral labeled bug reports that got fixed:", sum(labeled_df["label"] == "neutral"))
print("number of positive labeled bug reports that got fixed:",sum(labeled_df["label"] == "positive"))
print("number of negative labeled bug reports that got fixed:",sum(labeled_df["label"] == "negative"))

labeled_df_fixing_time = pd.merge(labeled_df, create_fixed[["bug_id","response_time(days)"]],left_on = "id", right_on="bug_id", how= "left").drop("bug_id",axis=1)
print(labeled_df_fixing_time)
print("average by labels: \n",labeled_df_fixing_time.groupby("label")["response_time(days)"].mean())

          id     label     score
1        117   neutral  0.912429
9        424   neutral  0.869095
16       838   neutral  0.544372
17       883   neutral  0.706331
19       909  positive  0.651654
...      ...       ...       ...
9974  108841   neutral  0.928398
9978  108862   neutral  0.528500
9983  108911   neutral  0.799396
9984  108912   neutral  0.611586
9985  108939   neutral  0.920898

[2938 rows x 3 columns]
number of neutral labeled bug reports that got fixed: 2234
number of positive labeled bug reports that got fixed: 38
number of negative labeled bug reports that got fixed: 666
          id     label     score  response_time(days)
0        117   neutral  0.912429                 1004
1        424   neutral  0.869095                  136
2        838   neutral  0.544372                  175
3        883   neutral  0.706331                  184
4        909  positive  0.651654                  169
...      ...       ...       ...                  ...
2933  108841   neutral  0

In [ ]:
print(df1_des["description"][:10])

0    created j daniel powell dan java linux org mon...
1    created thorsten heit thorsten wurzelausix cs ...
2    created dan mcguirk mcguirk indirect com thurs...
3    created keith adams kma cs brown edu thursday ...
4    created ralf hildebrandt r hildebrandt tu bs d...
5    created berkley shands berkley cs wustl edu we...
6    created berkley shands berkley cs wustl edu we...
7    created berkley shands berkley cs wustl edu we...
8    created berkley shands berkley cs wustl edu we...
9    created bert driehuis driehuis playbeing org f...
Name: description, dtype: object


In [ ]:
!pip  install lime


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283835 sha256=f2012d4508c36e9b485b03e59c15a73958b9a4b19db8a5073353a9f09a40c502
  Stored in directory: /root/.cache/pip/wheels/fd/a2/af/9ac0a1a85a27f314a06b39e1f492bee1547d52549a4606ed89
Successfully built lime


In [ ]:
import lime
import torch
import torch.nn.functional as F
from lime.lime_text import LimeTextExplainer

from transformers import AutoTokenizer, AutoModelForSequenceClassification

filename_model = 'cardiffnlp/twitter-roberta-base-sentiment-latest'
tokenizer = AutoTokenizer.from_pretrained(filename_model)
model = AutoModelForSequenceClassification.from_pretrained(filename_model)
# model = model.to(device)
class_names = ['positive','negative', 'neutral']

def predictor(texts):
    # tokens = tokenizer(texts, return_tensors="pt", padding=True).to(device)
    tokens = tokenizer(texts, return_tensors="pt", padding=True)
    outputs = model(**tokens)
    tensor_logits = outputs[0]
    probas = F.softmax(tensor_logits).detach().numpy()
    return probas

text = 'Building more bypasses will help the environment by reducing pollution and traffic jams in towns and cities.'
print(tokenizer(text, return_tensors='pt', padding=True))

explainer = LimeTextExplainer(class_names=class_names)
exp = explainer.explain_instance(text, predictor, num_features=20, num_samples=2000)
exp.show_in_notebook(text=text)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


{'input_ids': tensor([[    0, 37500,    55, 15037,   293,    40,   244,     5,  1737,    30,
          4881,  6631,     8,  1703, 28687,    11,  6815,     8,  1947,     4,
             2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [ ]:

import lime
import torch
import torch.nn.functional as F
from lime.lime_text import LimeTextExplainer

from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline,AutoTokenizer

index = df1_des.index[df1_des["bug_id"] == 73322].values[0]
id_73322_pos = df1_des["description"][index]
# id_46361_neu = df1_des[df1_des["bug_id"] == 46361]["description"].to_string()
# id_34295_neg = df1_des[df1_des["bug_id"] == 34295]["description"].to_string()


MODEL_NAME = "cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer_kwargs = {'padding':True,'truncation':True,'max_length':512}
model = pipeline(model=MODEL_NAME,tokenizer=tokenizer,device=0,**tokenizer_kwargs)
encoding = tokenizer(id_73322_pos)
tokens = model.tokenizer.tokenize(id_73322_pos)
inputs_512_tokens_string = model.tokenizer.convert_tokens_to_string(tokens)
print(inputs_512_tokens_string)

filename_model = 'cardiffnlp/twitter-roberta-base-sentiment-latest'
tokenizer = AutoTokenizer.from_pretrained(filename_model)
model = AutoModelForSequenceClassification.from_pretrained(filename_model)
# model = model.to(device)
class_names = ['positive','negative', 'neutral']

def predictor(texts):
    # tokens = tokenizer(texts, return_tensors="pt", padding=True).to(device)
    tokens = tokenizer(texts, return_tensors="pt", padding=True)
    outputs = model(**tokens)
    tensor_logits = outputs[0]
    probas = F.softmax(tensor_logits).detach().numpy()
    return probas

text = inputs_512_tokens_string
print(tokenizer(text, return_tensors='pt', padding=True))

explainer = LimeTextExplainer(class_names=class_names)
exp = explainer.explain_instance(text, predictor, num_features=20, num_samples=2000)
exp.show_in_notebook(text=text)